###### Machine Learning Engineer Nanodegree
## Supervised Learning
## Project: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: 

The given problem is a classification supervised learning problem. The two discrete categories to which the students can be assigned are 'Destined to Pass' and 'Needs Intervention' based on various (current) parameters. This should solve the purpose of the given problem. Regression learning caters to the situations with continuous output (for e.g. a real number) , which doesn't seem to be the case here. (On a side note, the given problem can also be viewed as regression problem, where we can output a number between 0 and 1, which represents the likelihood that the student will pass. Although that seems to be an overkill here)   

**

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
# TODO: Calculate number of students
n_students = len(student_data)

# TODO: Calculate number of features
n_features = len(student_data.columns) - 1 
## Corrected this after comments (excluding the label column)


# TODO: Calculate passing students
n_passed = len(student_data[student_data.passed == 'yes'])

# TODO: Calculate failing students
n_failed = len(student_data[student_data.passed == 'no'])

# TODO: Calculate graduation rate
grad_rate = float(n_passed) / n_students * 100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [4]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [5]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train , X_test , y_train , y_test = train_test_split(X_all , y_all , test_size=num_test , random_state=42)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])


Training set has 300 samples.
Testing set has 95 samples.


# Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer:  

Some observations about the data. 

- The number of training examples is pretty less (300)
- The number of features are moderate (>5) but not very large (<100) , 48 after adding dummy variables.
- The number of features is less than the nubmer of training examples.
- There is some level of correlation among the dummy variables produced (only one of them can hold value 1 for a given sample, the other will be 0). This might interfere with algorithms which assume conditional independence among features. 
- The data set is decently balanced (67 vs 33) among classes. 

The three models that I have chosen for this problem are :

### Naive Bayes 

#### Real World Application

   - Naive Bayes has been used a lot in natural language processing tasks like text classification , spam identification , sentiment analysis
   https://www.quora.com/In-what-real-world-applications-is-Naive-Bayes-classifier-used

#### Advantages

- Naive Bayes is extrememly fast, both during training and prediction. 
- Does not require a large number of training examples. 
- Easy to interpret the model.

#### Disadvantages

- Naive Bayes is based on the basic assumption that the features are conditionally independent of each other. 

#### Why I chose it 

- Given that we need a computationally cheap model which gives us good results fast, Naive Bayes fits the bill right away (although absolute conditional independence among features is missing). 
- It is also said to work better than others when the number of examples/samples is less. 
- The classes are not very imbalanced which is another requirement of Naive bayes. 



### Support Vector Machines (SVM)

#### Real World Application

Has been used for classification of Gene Expression Data. 

http://www.sciencedirect.com/science/article/pii/S1877050915004469


#### Advantages

- Works well even when the data is not linearly separable (by means of kernels) (We still do not know if the data is linearly separable or not). 
- Uses less storage for the results (only the support vectors). 
- Can be highly tuned via the regularization parameter (to avoid over fitting) and can be saved from outliers using the parameter C. 
- It delivers a unique solution to the optimization problem , local minima is not a problem.

#### Disadvantages

- Computationally expensive to train. 
- Can only be applied to a two class problem. 
- Model selection (kernel , reguralization parameter) plays a big part in the final success of the model (Can go either way). 
- Hard to interpret the model (makes it hard to explain to others)

#### Why I chose it

- Since we have a small number of training examples, the computation cost of SVM will not hurt us that much. Also we may be better off using less complex kernels (avoid RBF) or it might lead to overfitting. 
- Our problem is a classic two class supervised learning which SVM specialize in. 
- By fine tuning the model parameters later we can achieve some pretty good results out of SVM (while avoiding overfitting). 
- Again class imbalance is low which works in favor of SVM. 

https://stats.stackexchange.com/questions/24437/advantages-and-disadvantages-of-svm

### Adaboost (Using Decision Tree as the weak classifier)

#### Real World Application

- Has been used in pratical applications like Real Time Face Detection (Detect distinct human face in an image). 

https://arxiv.org/abs/1108.1353

#### Advantages

- Overcomes the problem of overfitting, as compared to when relying on a single decision tree. Hence improves classification accuracy over the single classifier. 
- Also does feature selection during training, resulting in a simpler model. 
- Similar to SVM, can capture non linear separation boundaries.
- Does not require a large number of parameters to tweak, hence simpler to implement. Advantage over SVM.
- Fast to implement (as compared to SVM)

#### Disadvantages

- Sensitive to noisy data and outliers.
- The solution is not optimal. 

#### Why I chose it

- With the advantage of capturing non linear separation boundaries , can be implemented faster and easily(less parameters to tweak) than SVM and results in a simpler model. So sort of a mid-way between time required and the complexity of the model. 

 
**

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [6]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [7]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A
# from sklearn import model_B
# from sklearn import model_C
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier 

# TODO: Initialize the three models
# EDIT (after comments - using random state wherever applicable)
clf_B = svm.SVC(random_state=42)
clf_A = GaussianNB()
clf_C = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(),random_state=42)

# TODO: Set up the training set sizes
# not reshuffling and resplitting the data as it 
# has been in a previous step
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train
y_train_300 = y_train

# TODO: Execute the 'train_predict' function for each classifier and each training set size
train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_A, X_train_300, y_train_300, X_test, y_test)

train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_B, X_train_300, y_train_300, X_test, y_test)

train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_C, X_train_300, y_train_300, X_test, y_test)

Training a GaussianNB using a training set size of 100. . .
Trained model in 0.0009 seconds
Made predictions in 0.0003 seconds.
F1 score for training set: 0.8467.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.8029.
Training a GaussianNB using a training set size of 200. . .
Trained model in 0.0008 seconds
Made predictions in 0.0003 seconds.
F1 score for training set: 0.8406.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.7244.
Training a GaussianNB using a training set size of 300. . .
Trained model in 0.0007 seconds
Made predictions in 0.0003 seconds.
F1 score for training set: 0.8038.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.7634.
Training a SVC using a training set size of 100. . .
Trained model in 0.0012 seconds
Made predictions in 0.0006 seconds.
F1 score for training set: 0.8777.
Made predictions in 0.0006 seconds.
F1 score for test set: 0.7746.
Training a SVC using a training set size of 200. . .
Trained model in 0.0027 seconds
Ma

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.


** Classifer 1 - Gaussian Naive Bayes**  

| Training Set Size | Training Time (msec) | Prediction Time (test)<br>(msec) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        .9               |          .2              |         .8467         |       .8029   |
| 200               |     .8            |               .2         |       .8406           |        .7244         |
| 300               |          .8               |             .2           |     .8038             |     .7634     |


** Classifer 2 - Support Vector**  

| Training Set Size | Training Time (msec) | Prediction Time (test)<br> (msec) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         1.2       |       .6    |    .8777         |        .7746    |
| 200               |        2.7          |         .9               |          .8679        |        .7815         |
| 300               |           5.9              |             1.3           |       .8761           |    .7838      |


** Classifer 3 - Decision Tree**  

| Training Set Size | Training Time (msec) | Prediction Time (test)<br>(msec) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |            2.0             |        .2                |       1           |          .6552       |
| 200               |         2.7               |            .5            |         1         |     .7402            |
| 300               |        3.9                 |               .3         |            1      |           .6102      |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer:

I have chosen Support Vector Machines as the learning algorithm for this problem as they outperform the other models (F1-score on testing data), particularly when the training size increases. Although, in similar cases with limited resources, cost, small sample size, usually Naive Bayes is the chosen strategy. As indicated by the training and prediction timings, Naive Bayes is still ahead of SVM and Adaboost. However, the gain is not an order gain. All the times for all the predictors are still limited to milliseconds. It does seem that with larger training size, the performance of both Naive Bayes and AdaBoost decreases on the test set, which indicates they are more prone to overfitting and don't generalize well in our particular problem. During prediction, all the models are pretty fast. 


**

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: 

![title](./svm.jpg "SVM")


The idea can best be explained by looking at the figure above. The students in our case represents the points. Each student belongs to one of the two classes 'Will Pass' (Boxes) or 'Needs Intervention' (Circle) . Let us simplify the problem by saying we have only two features 'Feature A' (say Education Status of the Parents) and 'Feature B' (say travel time to the school). For this case we can easily plot the points in a two dimensional space (where the axis represents the two features in a numerical form). By looking at the graph it is clear to us that the two classes form  pretty distinct clusters (proximity of points belonging to similar classes). Support Vector Machine algorithm tries to find a line in this space which separates (runs in the middle) the points belonging to the two classes. Of all the possible lines which perform the desired task, SVM tries to find the line which has the 'maximum margin' from both the classes. This implies that the algorithm tries to maximize the distance of the line to the closest point on either side. As we can see even though Line B and Line C have a great distance from the closest point of one class, they are pretty close to the closest point of the other class. Line A on the other hand , chooses the middle ground , and has almost equal distance from the closest point on both the sides and thus maximizes the margins if we just consider the three lines. So if the SVM algorithm had to choose between the three lines it will choose Line A. When we get a fresh point (student) , SVM based on its feature values determines on which side of the line the point lies and accordingly classifies the student as to whether the student is 'Destined to Pass' or 'Needs Intevention'

We can take the same problem to a higher dimension (more features, just as we have in our problem) and the SVM algorithm will try and find the 'maximum margin' separating hyper-plane rather than a line.  The maximum margin criteria works well since by selecting the line with maximum margins we are insuring ourselves against outliers and noise (Points too far to the line don't matter in the final boundary). Also, usually the SVM's work with soft margins i.e. we ignore certain points which are either too close to the line or are on the wrong side of the line while deciding our margins.  The points which determine the margins on both the sides of the line are called support vectors and hence the name of the algorithm. Support vectors can also help us in determining non-linear boundaries. We can transform all the points (uniformly) to another space where the points are linearly separable. Find a separating line in the new space. For any new point, we can transform it similarly to the new space, and depending on which side of the line the point lies, classify it.

So whenever we get a new student data, we can feed it to the SVM Model. Based on the class which the classfier predicts for the student, 'Destined to Pass' vs 'Needs Intervention' , we can take further action. The classifier is supposed to give the prediction in a reasonable amount of time (order of milliseconds), and also the storage required by the final model is less (it only stores a linear combination of the support vectors - which are fewer in number as compared to the data points). 

**

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [8]:
from sklearn import svm
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.cross_validation import StratifiedShuffleSplit


# TODO: Import 'GridSearchCV' and 'make_scorer'

# TODO: Create the parameters list you wish to tune
parameters = { 'kernel':('linear' , 'rbf') , 'C':[1, 10 , 100 , 1000] , 'gamma':[ 0.01 , 0.001 , 0.0001] }

# TODO: Initialize the classifier
clf = clf_B

cv = StratifiedShuffleSplit(y_train , n_iter=50 , random_state=42)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label="yes")

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf,  param_grid=parameters, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train , y_train)

print grid_obj.get_params()

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

{'n_jobs': 1, 'verbose': 0, 'estimator__gamma': 'auto', 'estimator__decision_function_shape': None, 'estimator__probability': False, 'param_grid': {'kernel': ('linear', 'rbf'), 'C': [1, 10, 100, 1000], 'gamma': [0.01, 0.001, 0.0001]}, 'cv': None, 'scoring': make_scorer(f1_score, pos_label=yes), 'estimator__cache_size': 200, 'estimator__verbose': False, 'pre_dispatch': '2*n_jobs', 'estimator__kernel': 'rbf', 'fit_params': {}, 'estimator__max_iter': -1, 'refit': True, 'iid': True, 'estimator__shrinking': True, 'estimator__degree': 3, 'estimator__class_weight': None, 'estimator__C': 1.0, 'estimator__random_state': 42, 'estimator': SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=42, shrinking=True,
  tol=0.001, verbose=False), 'estimator__coef0': 0.0, 'error_score': 'raise', 'estimator__tol': 0.001}
Made predictions in 0.0036 seconds.
Tuned model has a training F1 s

### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: 

The final F1 score for the tuned model is .8323 (as compared to .8761) and .7945 for test (as compared to .7838). So we can say the model is generalizing better (less overfitting) after the tuning. 

**

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.